In [24]:
from sklearn.base import BaseEstimator, TransformerMixin

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/rohlik-sales-forecasting-challenge-v2/calendar.csv
/kaggle/input/rohlik-sales-forecasting-challenge-v2/test_weights.csv
/kaggle/input/rohlik-sales-forecasting-challenge-v2/inventory.csv
/kaggle/input/rohlik-sales-forecasting-challenge-v2/sales_train.csv
/kaggle/input/rohlik-sales-forecasting-challenge-v2/sales_test.csv
/kaggle/input/rohlik-sales-forecasting-challenge-v2/solution.csv


In [9]:
# Load data into DataFrames
calendar = pd.read_csv("/kaggle/input/rohlik-sales-forecasting-challenge-v2/calendar.csv")
test_weights = pd.read_csv("/kaggle/input/rohlik-sales-forecasting-challenge-v2/test_weights.csv")
inventory = pd.read_csv("/kaggle/input/rohlik-sales-forecasting-challenge-v2/inventory.csv")
sales_train = pd.read_csv("/kaggle/input/rohlik-sales-forecasting-challenge-v2/sales_train.csv")
sales_test = pd.read_csv("/kaggle/input/rohlik-sales-forecasting-challenge-v2/sales_test.csv")
solution = pd.read_csv("/kaggle/input/rohlik-sales-forecasting-challenge-v2/solution.csv")

# Display the first few rows of each DataFrame
'''for name, df in zip(["calendar", "test_weights", "inventory", "sales_train", "sales_test", "solution"], 
                     [calendar, test_weights, inventory, sales_train, sales_test, solution]):
    print(f"{name} DataFrame:")
    print(df.head(), "\n")'''

'for name, df in zip(["calendar", "test_weights", "inventory", "sales_train", "sales_test", "solution"], \n                     [calendar, test_weights, inventory, sales_train, sales_test, solution]):\n    print(f"{name} DataFrame:")\n    print(df.head(), "\n")'

# Load data

In [10]:
#i define helper functions
def get_intersection(sales_train,df_names,dfs):
    for df,name in zip(dfs,df_names):
        common_columns = sales_train.columns.intersection(df.columns)
        print(name)
        print(common_columns)
        print(" ")
    

In [11]:
# i want to merge the calendar and inventory dataframes to the sales_train dataframe
#i need to identify the common columns and merge on those
get_intersection(sales_train,['calendar','inventory','sales_train'],[calendar,inventory,sales_test])

calendar
Index(['date', 'warehouse'], dtype='object')
 
inventory
Index(['unique_id', 'warehouse'], dtype='object')
 
sales_train
Index(['unique_id', 'date', 'warehouse', 'total_orders', 'sell_price_main',
       'type_0_discount', 'type_1_discount', 'type_2_discount',
       'type_3_discount', 'type_4_discount', 'type_5_discount',
       'type_6_discount'],
      dtype='object')
 


In [12]:
sales_train = pd.merge(sales_train, calendar, on=['date','warehouse'], how='left')
sales_train = pd.merge(sales_train, inventory, on=['unique_id','warehouse'], how='left')

In [13]:
sales_test = pd.merge(sales_test, calendar, on=['date','warehouse'], how='left')
sales_test = pd.merge(sales_test, inventory, on=['unique_id','warehouse'], how='left')

In [14]:
#In these problems it is best to use the date as index
sales_train['date']=pd.to_datetime(sales_train['date'])
sales_train.set_index('date',inplace=True)
sales_test['date']=pd.to_datetime(sales_test['date'])
sales_test.set_index('date',inplace=True)

In [19]:
#i need to drop the featurees that are in the train dataset but not in the test one 
unique_columns = sales_train.columns.symmetric_difference(sales_test.columns)
print(unique_columns)
sales_train.drop('availability', axis=1, inplace=True)

Index(['availability', 'sales'], dtype='object')


# EDA

In [ ]:
#i skip this part

# data preprocessing 

In [21]:
#for data preprocessing i convert all the object features to categorical variables
#i identif and separate the categorical and non categorical features
# Separate numeric and non-numeric columns
numeric_cols = sales_train.select_dtypes(include=['number']).columns.tolist()
non_numeric_cols = sales_train.select_dtypes(exclude=['number']).columns.tolist()
print(numeric_cols)
print(non_numeric_cols)
print(sales_train[non_numeric_cols].dtypes)

['unique_id', 'total_orders', 'sales', 'sell_price_main', 'type_0_discount', 'type_1_discount', 'type_2_discount', 'type_3_discount', 'type_4_discount', 'type_5_discount', 'type_6_discount', 'holiday', 'shops_closed', 'winter_school_holidays', 'school_holidays', 'product_unique_id']
['warehouse', 'holiday_name', 'name', 'L1_category_name_en', 'L2_category_name_en', 'L3_category_name_en', 'L4_category_name_en']
warehouse              object
holiday_name           object
name                   object
L1_category_name_en    object
L2_category_name_en    object
L3_category_name_en    object
L4_category_name_en    object
dtype: object


In [22]:
sales_train[non_numeric_cols] = sales_train[non_numeric_cols].astype('category')

In [25]:
# i want to add some new features (eg the year and the sinusoidal year)

class DateFeatureTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        
        # Ensure the index is a DatetimeIndex
        if not isinstance(X.index, pd.DatetimeIndex):
            raise ValueError("Index must be a DatetimeIndex")
        
        # Extract basic time features
        X["year"] = X.index.year
        X["month"] = X.index.month
        X["week"] = X.index.isocalendar().week
        
        # Cyclic encoding for month
        X["month_sin"] = np.sin(2 * np.pi * X["month"] / 12)
        X["month_cos"] = np.cos(2 * np.pi * X["month"] / 12)
        
        # Cyclic encoding for week
        X["week_sin"] = np.sin(2 * np.pi * X["week"] / 52)
        X["week_cos"] = np.cos(2 * np.pi * X["week"] / 52)

        return X


In [26]:
# Example usage:
transformer = DateFeatureTransformer()
sales_train = transformer.fit_transform(sales_train)

In [28]:
# Example usage:
transformer = DateFeatureTransformer()
sales_test = transformer.fit_transform(sales_test)

In [27]:
sales_train.head()

,unique_id,warehouse,total_orders,sales,sell_price_main,type_0_discount,type_1_discount,type_2_discount,type_3_discount,type_4_discount,...,L2_category_name_en,L3_category_name_en,L4_category_name_en,year,month,week,month_sin,month_cos,week_sin,week_cos
date,,,,,,,,,,,,,,,,,,,,,
2024-03-10,4845,Budapest_1,6436.0,16.34,646.26,0.00000,0.0,0.0,0.0,0.15312,...,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,2024,3,10,1.000000e+00,6.123234e-17,0.935016,0.354605
2021-05-25,4845,Budapest_1,4663.0,12.63,455.96,0.00000,0.0,0.0,0.0,0.15025,...,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,2021,5,21,5.000000e-01,-8.660254e-01,0.568065,-0.822984
2021-12-20,4845,Budapest_1,6507.0,34.55,455.96,0.00000,0.0,0.0,0.0,0.15025,...,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,2021,12,51,-2.449294e-16,1.000000e+00,-0.120537,0.992709
2023-04-29,4845,Budapest_1,5463.0,34.52,646.26,0.20024,0.0,0.0,0.0,0.15312,...,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,2023,4,17,8.660254e-01,-5.000000e-01,0.885456,-0.464723
2022-04-01,4845,Budapest_1,5997.0,35.92,486.41,0.00000,0.0,0.0,0.0,0.15649,...,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,2022,4,13,8.660254e-01,-5.000000e-01,1.0,-0.0


In [30]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
import pandas as pd

# Identify numeric and categorical columns
# Exclude 'sales' from numeric columns
numeric_cols = [col for col in sales_train.select_dtypes(include=['number']).columns if col != "sales"]
non_numeric_cols = sales_train.select_dtypes(exclude=['number']).columns.tolist()
print(numeric_cols)
print(non_numeric_cols)

# Define preprocessing steps
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="constant", fill_value=-1)),  # Fill NA with -1
    ("scaler", StandardScaler())  # Normalize numeric features
])

categorical_transformer = Pipeline(steps=[
    ("encoder", OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1))  # Encode categorical features
])

# Combine into a ColumnTransformer
preprocessor = ColumnTransformer(transformers=[
    ("num", numeric_transformer, numeric_cols),
    ("cat", categorical_transformer, non_numeric_cols)
])


['unique_id', 'total_orders', 'sell_price_main', 'type_0_discount', 'type_1_discount', 'type_2_discount', 'type_3_discount', 'type_4_discount', 'type_5_discount', 'type_6_discount', 'holiday', 'shops_closed', 'winter_school_holidays', 'school_holidays', 'product_unique_id', 'year', 'month', 'week', 'month_sin', 'month_cos', 'week_sin', 'week_cos']
['warehouse', 'holiday_name', 'name', 'L1_category_name_en', 'L2_category_name_en', 'L3_category_name_en', 'L4_category_name_en']


# train an LGBM model on 

In [32]:
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from lightgbm import LGBMRegressor

# Define target and features
y = sales_train["sales"]
X = sales_train.drop(columns=["sales"])

# Define TimeSeriesSplit (test window = 14 days)
tscv = TimeSeriesSplit(n_splits=5, test_size=14)  # Adjust n_splits based on your data size

# Initialize LGBM model
model = LGBMRegressor(n_estimators=100, learning_rate=0.1, random_state=42)

# Impute missing values in y
y_imputer = SimpleImputer(strategy="mean")  # Options: "mean", "median", "constant", etc.
y = pd.Series(y_imputer.fit_transform(y.values.reshape(-1, 1)).flatten(), index=y.index)

# Apply cross-validation
cv_scores = cross_val_score(model, preprocessor.fit_transform(X), y, cv=tscv, scoring="neg_mean_absolute_error")

# Print results
print("Cross-validation MAE scores:", -cv_scores)
print("Mean MAE:", -np.mean(cv_scores))


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.342496 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2889
[LightGBM] [Info] Number of data points in the train set: 4007349, number of used features: 29
[LightGBM] [Info] Start training from score 108.381566
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.335463 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2890
[LightGBM] [Info] Number of data points in the train set: 4007363, number of used features: 29
[LightGBM] [Info] Start training from score 108.381408
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.319911 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is n